In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
link = "./rotten_tomatoes_critic_reviews_tokens.csv"
link2 ='./rotten_tomatoes_movies.csv'

critic_reviews = pd.read_csv(link, converters={'review_date': pd.to_datetime})
movies = pd.read_csv(link2, converters={'original_release_date': pd.to_datetime, 'streaming_release_date': pd.to_datetime})


# Normalize the score for each review on 100

In [4]:
# method to convert format rate 3/5 to float
def get_score(x):
    x = x.replace(" ", "")
    letters = {"A":14, "B":11, "C":8,"D":5,"E":2, "F":0}
        #convert string "3.4/5" or "67/100" to rate on 100
    if (type(x)==str) and ('/' in x) and (float(x.split("/")[1]) > float(x.split("/")[0])):
        return 100*(float(x.split("/")[0]) / float(x.split("/")[1]))
    elif x[0] in letters.keys():
        #convert letters rate to rate on 100
        return 100*((letters[x[0]] + 44 - ord(x[1]) if len(x)>1 and x[1] in "-+" else letters[x[0]])/15)


critic_reviews["review_score_float"] = critic_reviews[critic_reviews["review_score"].isna()==False]["review_score"].apply(get_score)
critic_reviews[critic_reviews["review_score_float"].isna()==False]["review_score_float"]

3          70.000000
6          25.000000
7          70.000000
8          73.333333
9          60.000000
             ...    
1130012    40.000000
1130013    70.000000
1130014    80.000000
1130015    70.000000
1130016    53.333333
Name: review_score_float, Length: 777569, dtype: float64

In [5]:
critic_reviews.describe()

,Unnamed: 0,review_score_float
count,1.130017e+06,777569.000000
mean,5.650080e+05,61.542096
std,3.262080e+05,19.459735
min,0.000000e+00,0.000000
25%,2.825040e+05,50.000000
50%,5.650080e+05,62.500000
75%,8.475120e+05,80.000000
max,1.130016e+06,99.900000


# Merge audience score with review table

In [12]:
#movies.columns

In [11]:
#critic_reviews.columns

In [13]:
critic_reviews = critic_reviews.merge(movies[["rotten_tomatoes_link", "audience_rating"]], on="rotten_tomatoes_link")

In [17]:
critic_reviews.loc[~critic_reviews['review_score_float'].isna()].head()

,Unnamed: 0,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,review_score_float,tokens,audience_rating
3,3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...,70.000000,"['whether', 'audiences', 'will', 'get', 'behin...",53.0
6,6,m/0814255,Nick Schager,False,Slant Magazine,Rotten,1/4,2010-02-10,Harry Potter knockoffs don't come more transpa...,25.000000,"['harry', 'potter', 'knockoffs', 'do', ""n't"", ...",53.0
7,7,m/0814255,Bill Goodykoontz,True,Arizona Republic,Fresh,3.5/5,2010-02-10,"Percy Jackson isn't a great movie, but it's a ...",70.000000,"['percy', 'jackson', 'is', ""n't"", 'a', 'great'...",53.0
8,8,m/0814255,Jordan Hoffman,False,UGO,Fresh,B,2010-02-10,"Fun, brisk and imaginative",73.333333,"['fun', ',', 'brisk', 'and', 'imaginative']",53.0
9,9,m/0814255,Jim Schembri,True,The Age (Australia),Fresh,3/5,2010-02-10,"Crammed with dragons, set-destroying fights an...",60.000000,"['crammed', 'with', 'dragons', ',', 'set-destr...",53.0


I created a new column with a calcul of the difference of audience and critic convert to absolute value then convert as %.
To be able to compare more I substract with  1. In this way the highest is the score more closer the magasin is with the audience rating

In [27]:
critic_reviews["difference_rating_audience_vs_critics"] = 1 - (critic_reviews["audience_rating"] - critic_reviews["review_score_float"]).abs()/100


In [28]:
critic_reviews["difference_rating_audience_vs_critics"].describe()

count    774953.000000
mean          0.842579
std           0.127166
min           0.040000
25%           0.770000
50%           0.870000
75%           0.940000
max           1.000000
Name: difference_rating_audience_vs_critics, dtype: float64

In [56]:
table_pivot = pd.pivot_table(critic_reviews, values=["difference_rating_audience_vs_critics"], index=['publisher_name'], aggfunc=['mean', "std", "count"])
table_pivot.sort_values([('mean','difference_rating_audience_vs_critics'),('std', 'difference_rating_audience_vs_critics'),('count', 'difference_rating_audience_vs_critics')], ascending=[0,1,0]).head(10)

,mean,std,count
,difference_rating_audience_vs_critics,difference_rating_audience_vs_critics,difference_rating_audience_vs_critics
publisher_name,,,
Polygon,1.000000,0.000000,2
Women Rocking Hollywood,1.000000,NaN,1
Horror Squad,0.990000,0.000000,2
Cinegarage,0.990000,NaN,1
Mansized,0.990000,NaN,1
doghatesfilm.com,0.990000,NaN,1
Odyssey,0.986667,NaN,1
White Plains Times,0.983333,NaN,1


In [52]:
table_pivot.describe()

,mean,std,count
,difference_rating_audience_vs_critics,difference_rating_audience_vs_critics,difference_rating_audience_vs_critics
count,1607.000000,1475.000000,2230.000000
mean,0.835683,0.121029,347.512556
std,0.061650,0.036163,923.806485
min,0.240000,0.000000,0.000000
25%,0.819791,0.106487,0.000000
50%,0.843393,0.120554,18.000000
75%,0.863789,0.136908,228.000000
max,1.000000,0.341223,8487.000000


In [53]:
table_pivot.columns

MultiIndex([( 'mean', 'difference_rating_audience_vs_critics'),
            (  'std', 'difference_rating_audience_vs_critics'),
            ('count', 'difference_rating_audience_vs_critics')],
           )

I select 25% of magasins that have a count of difference > of quantile .75

In [60]:

#critic_reviews[('count', 'difference_rating_audience_vs_critics')].quantile(.75)
table_pivot_2 = table_pivot[table_pivot[('count','difference_rating_audience_vs_critics')] > table_pivot.quantile(.75)[('count','difference_rating_audience_vs_critics')]]
#table_pivot.sort_values([('mean','difference_rating_audience_vs_critics'),('std', 'difference_rating_audience_vs_critics'),('count', 'difference_rating_audience_vs_critics')], ascending=[0,1,0]).head(10)

,mean,std,count
,difference_rating_audience_vs_critics,difference_rating_audience_vs_critics,difference_rating_audience_vs_critics
publisher_name,,,
2UE That Movie Show,0.848530,0.126029,347
3AW,0.837290,0.123461,1347
3BlackChicks Review,0.817244,0.152270,254
7M Pictures,0.837927,0.128743,1404
AARP Movies for Grownups,0.860712,0.112904,365
...,...,...,...
sbs.com.au,0.866654,0.104030,645
sbs.is,0.848034,0.118623,435
